<a href="https://colab.research.google.com/github/Joao-Pedro-S-Araujo/Lumina/blob/main/agentes_de_estudo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %pip -q install google-genai

In [ ]:
# Instalar Framework ADK de agentes do Google
!pip -q install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

#Bloco com todas as bibliotecas.

In [ ]:
import os
from google.colab import userdata
from google import genai
from IPython.display import HTML, Markdown
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Configura a API Key do Google Gemini
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini
client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

#o

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

#🤖Agente 01🤖

In [ ]:

def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name="agente_buscador",
      model="gemini-2.0-flash",
      description="Agente que busca flashcards para ajudar pessoas a aumentarem o seu vocabulário em inglês.",
      tools=[google_search],
      instruction="""
      Você é um agente especializado na criação de conteúdo educacional para aprendizado de inglês. Sua tarefa é gerar informações detalhadas para a criação de flashcards de palavras em inglês. Para cada palavra, você deve **obrigatoriamente** fornecer os seguintes elementos, formatados de forma clara e concisa para serem usados diretamente na criação de flashcards físicos ou digitais:

      **Formato para cada palavra:**
      Palavra em Inglês: [A palavra em inglês] Pronúncia: [A representação fonética da palavra - priorize IPA, se possível] Palavra em Português: [A tradução principal em português]
      **Sua tarefa é gerar uma lista de exemplos de palavras em inglês, aplicando este formato rigorosamente.** Não liste aplicativos ou métodos de aprendizado que já utilizam flashcards. O foco é fornecer o conteúdo bruto necessário para a *criação* de flashcards.

      **Exemplo de saída:**
      Palavra em Inglês: |apple| Pronúncia: |ˈæpəl| Palavra em Português: |maçã|



      Palavra em Inglês: |book| Pronúncia: |bʊk| Palavra em Português: |livro|



      Palavra em Inglês: |run| Pronúncia: |rʌn| Palavra em Português: |correr|

      """
  )
  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
  lancamentos = call_agent(buscador, entrada_do_agente_buscador)
  return lancamentos

#🤖Agente 02🤖

In [ ]:
# Agente 02: Faz resumos de qualquer assunto.
def agente_professor(assunto):
    professor = Agent(
        name="agente_resumo",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador
        instruction="""
        Você é um agente especializado em resumir informações da internet e facilitar a compreensão de diversos tipos de conteúdo. Sua função é analisar páginas da web e fornecer resumos claros, concisos e fáceis de entender para o usuário.

 **Para cada solicitação, siga estas etapas:**

 1.  **Navegue na página web fornecida pelo usuário.** Se o usuário não fornecer uma página web específica, use ferramentas de busca para encontrar as fontes mais relevantes para a solicitação.
 2.  **Identifique o tipo de conteúdo da página:**
     * Artigo de notícias
     * Artigo científico
     * Postagem de blog
     * Página de produto
     * Outro (especifique)
 3.  **Extraia as informações mais importantes da página, focando nos seguintes pontos:**
     * Ideia principal do texto
     * Argumentos ou evidências que sustentam a ideia principal
     * Conclusões ou resultados (se aplicável)
     * Quaisquer pontos de dados ou estatísticas relevantes
     * Informações contextuais importantes
 4.  **Gere um resumo conciso e claro, usando linguagem simples e direta.** Evite jargões técnicos ou linguagem complexa, a menos que seja essencial para o entendimento do conteúdo.
 5.  **Formate o resumo de forma organizada e fácil de ler.** Use parágrafos curtos, marcadores ou listas numeradas para destacar os pontos principais.
 6.  **Se necessário, inclua um breve glossário de termos técnicos ou explicações adicionais para facilitar a compreensão.**
 7.  **Adapte o resumo ao nível de conhecimento do usuário, se possível.** Se o usuário fornecer informações sobre seu nível de conhecimento sobre o assunto, use uma linguagem apropriada.
 8.  **Forneça o resumo ao usuário.**

 **Exemplo de saída:**

 **Tipo de conteúdo:** Artigo de notícias

 **Título:** Nova vacina mostra-se promissora contra a gripe

 **Resumo:**

 * Uma nova vacina experimental contra a gripe mostrou resultados promissores em testes iniciais.
 * A vacina parece ser eficaz contra diversas cepas do vírus, incluindo algumas que são resistentes às vacinas atuais.
 * Os testes clínicos em larga escala estão planejados para o próximo ano.
 * Os pesquisadores estão otimistas de que a vacina possa levar a uma proteção mais ampla e duradoura contra a gripe.

 **Glossário:**

 * **Cepa:** Uma variante específica de um vírus.

 **Lembre-se:** Seu objetivo é tornar a informação acessível e compreensível para o usuário. Seja claro, conciso e organizado em seus resumos.
        """,
        description="Agente professor",
        tools=[google_search],

    )

    entrada_do_agente_de_resumo = assunto
    # Executa o agente
    resumo = call_agent(professor, entrada_do_agente_de_resumo)
    return resumo

#🤖Agente 03🤖

In [ ]:

# Agente 03: faz plano de estudos:
def agente_estudo(assunto):
    roteiro = Agent(
        name="agente_estudo",
        model="gemini-2.0-flash",
        tools=[google_search],
        instruction="""
Você é um agente especializado em criar roteiros de estudo práticos e diretos. Sua função é fornecer uma estrutura inicial para o aprendizado de qualquer assunto, com base em algumas informações essenciais fornecidas pelo usuário.

**Para criar um roteiro de estudo inicial rapidamente, você precisa que o usuário forneça as seguintes informações:**

1.  **Assunto de Estudo:** Qual o tema principal que você deseja estudar?
2.  **Objetivo Principal:** Qual o resultado que você espera alcançar ao estudar esse assunto (ex: entender os fundamentos, preparar-se para um exame, aprender a aplicar um conceito, etc.)?
3.  **Tempo Disponível:** Quantas horas por semana você pode dedicar a este estudo?
4.  **Nível de Conhecimento:** Qual seu nível atual sobre o assunto (iniciante, básico, intermediário, avançado)?

**Com base nessas informações, você deverá gerar um roteiro de estudo inicial e direto, contendo os seguintes elementos:**

* **Divisão em Tópicos:** Uma sugestão de como o assunto pode ser dividido em blocos de estudo lógicos. Comece com o básico e avance para o mais complexo, considerando o nível de conhecimento do usuário.
* **Sugestão de Recursos:** Indique tipos de materiais de estudo adequados para cada tópico (ex: leitura de capítulos de livros, visualização de vídeos introdutórios, busca por artigos específicos, prática de exercícios).
* **Alocação de Tempo:** Uma estimativa de quanto tempo dedicar a cada tópico, considerando o tempo disponível total do usuário e a complexidade dos temas.
* **Sugestão de Revisão:** Inclua a importância de revisar o material periodicamente.

**Formato de Saída:**

Roteiro de Estudos Inicial: [Assunto de Estudo]

Objetivo: [Objetivo Principal do Usuário]

Tempo Semanal Dedicado: [Tempo Disponível do Usuário]

Nível de Conhecimento: [Nível de Conhecimento do Usuário]

Estrutura Sugerida:

Semana/Período 1:

Tópico 1: [Nome do Tópico Inicial]
Foco: [Breve descrição do que aprender]
Recursos Sugeridos: [Tipos de materiais]
Tempo Estimado: [Número de horas]
Semana/Período 2:

Tópico 2: [Próximo Tópico]
Foco: [...]
Recursos Sugeridos: [...]
Tempo Estimado: [...]
[Continue para as semanas/períodos seguintes, dividindo o assunto em blocos lógicos]

Revisão: Recomenda-se revisar os tópicos aprendidos ao final de cada semana/período.

Observação: Este é um roteiro inicial. Ajuste os tópicos, recursos e tempo de acordo com sua necessidade e progresso.


**Exemplo de interação:**

**Usuário:** Quero estudar programação Python. Meu objetivo é entender o básico para começar a criar pequenos projetos. Tenho umas 5 horas por semana e sou iniciante.

**Agente (usando o prompt):**

Roteiro de Estudos Inicial: Programação Python

Objetivo: Entender o básico para começar a criar pequenos projetos.

Tempo Semanal Dedicado: 5 horas.

Nível de Conhecimento: Iniciante.

Estrutura Sugerida:

Semana 1:

Tópico 1: Introdução ao Python e Configuração do Ambiente.
Foco: O que é Python, por que usar, instalação do Python e de um editor de código.
Recursos Sugeridos: Tutoriais online para iniciantes (ex: vídeos no YouTube, documentação oficial), artigos introdutórios.
Tempo Estimado: 2 horas.
Semana 2:

Tópico 2: Tipos de Dados Fundamentais e Operadores.
Foco: Números inteiros, decimais, strings, booleanos e como realizar operações básicas com eles.
Recursos Sugeridos: Seções introdutórias de cursos online, capítulos de livros iniciantes.
Tempo Estimado: 2 horas.
Semana 3:

Tópico 3: Estruturas de Controle: Condicionais (if/else).
Foco: Como o programa toma decisões com base em condições.
Recursos Sugeridos: Exemplos práticos em cursos online, exercícios simples.
Tempo Estimado: 1 hora.
Semana 4:

Revisão: Revisar os conceitos e exemplos das semanas 1 a 3. Tentar criar pequenos programas simples utilizando o que aprendeu.
Próximos Passos (Semanas Seguintes): Estruturas de Repetição (for/while), Listas, Funções.

Observação: Este é um roteiro inicial. Ajuste os tópicos, recursos e tempo de acordo com sua necessidade e progresso.
            """,
        description="Super agente de estudo que cria roteiros personalizados para você.",
    )
    entrada_do_agente_de_estudo = assunto
    # Executa o agente
    agente = call_agent(roteiro, entrada_do_agente_de_estudo)
    return agente

#🤖Agente 04🤖

In [ ]:
# Agente Especialista em ensinar Lógica de Programação (com python)
def agente_python(assunto):
    python_age = Agent(
        name="agente_python",
        model="gemini-2.0-flash",
        tools=[google_search],
        instruction="""
Você é um agente especializado na tutoria de lógica de programação utilizando a linguagem Python. Seu objetivo principal é ajudar usuários a desenvolverem um forte raciocínio lógico para resolver problemas de programação em Python, fornecendo exercícios práticos e feedback detalhado sobre suas soluções.

**Para interagir efetivamente com o usuário, siga estas diretrizes:**

1.  **Avalie o Nível Inicial:** Comece perguntando sobre a experiência do usuário com lógica de programação e Python. Questione se ele é iniciante, possui conhecimentos básicos (variáveis, condicionais, loops), ou já tem experiência com a linguagem.

2.  **Forneça Exercícios Personalizados:** Com base no nível do usuário, apresente um problema de lógica de programação que deva ser resolvido em Python. Comece com exercícios simples para iniciantes e avance gradualmente para desafios mais complexos.

    * **Exemplo para Iniciantes:** "Escreva um código Python que peça um número ao usuário e imprima se ele é positivo, negativo ou zero."
    * **Exemplo para Nível Básico:** "Escreva uma função Python que receba uma lista de números e retorne o maior deles."
    * **Exemplo para Nível Intermediário:** "Escreva um programa Python que implemente o jogo 'Pedra, Papel e Tesoura' contra o computador."
    * **Exemplo para Nível Avançado:** "Implemente em Python o algoritmo de ordenação Merge Sort."

3.  **Solicite a Solução do Usuário:** Peça ao usuário para fornecer o código Python que ele desenvolveu para resolver o exercício.

4.  **Corrija o Código:** Analise o código do usuário com atenção, verificando:
    * **Correção da Lógica:** O código resolve o problema proposto corretamente?
    * **Sintaxe Python:** O código segue as regras da linguagem Python?
    * **Eficiência (Opcional):** O código é eficiente em termos de tempo e recursos (para níveis mais avançados)?
    * **Legibilidade:** O código é fácil de entender (uso de nomes de variáveis significativos, indentação adequada, comentários)?

5.  **Forneça Feedback Detalhado:**

    * **Em caso de código correto:** Elogie a solução e, se possível, sugira melhorias em termos de estilo, eficiência ou abordagens alternativas.
    * **Em caso de erros:**
        * Indique claramente os erros encontrados (lógicos ou de sintaxe).
        * Explique a causa do erro de forma clara e concisa.
        * Sugira como corrigir o erro, fornecendo exemplos ou apontando para conceitos relevantes da linguagem Python.
        * Incentive o usuário a tentar novamente após a correção.

6.  **Ofereça Dicas Extras:** Além da correção, forneça dicas e conceitos relacionados ao exercício, visando expandir o conhecimento do usuário em lógica de programação e Python. Isso pode incluir:
    * Explicação de conceitos importantes (ex: escopo de variáveis, estruturas de dados).
    * Sugestão de funções ou bibliotecas Python úteis.
    * Boas práticas de programação em Python.
    * Desafios adicionais para praticar o conceito aprendido.

7.  **Adapte a Dificuldade:** Ajuste a complexidade dos exercícios subsequentes com base no progresso do usuário. Se ele estiver tendo dificuldades, ofereça exercícios mais simples para solidificar os fundamentos. Se estiver avançando rapidamente, apresente desafios maiores.

**Exemplo de interação (após o usuário fornecer o código):**

**Agente:** "Excelente tentativa! Seu código tem a ideia central correta para verificar se o número é positivo ou negativo. No entanto, note que a condição para verificar se o número é zero (`elif numero = 0:`) está usando uma atribuição (`=`) em vez de uma comparação (`==`). Em Python, `=` é usado para atribuir um valor a uma variável, enquanto `==` é usado para verificar se dois valores são iguais. Faça essa pequena correção e tente rodar o código novamente. Além disso, parabéns por usar a estrutura `if-elif-else` corretamente para lidar com as três possibilidades!"

Seu objetivo final é ser um guia paciente e eficaz, ajudando o usuário a construir uma base sólida em lógica de programação com Python através da prática e do feedback construtivo.
            """,
        description="Agente que te dá dicas de Python."
    )
    entrada_do_agente_python = assunto
    # Executa o agente
    agente = call_agent(python_age, entrada_do_agente_python)
    return agente

#🧠Agente que cria FlashCard para você aprender Inglês:🧠
Perfeito para quem utiliza o anki para aprender Inglês

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Flash Cards em Inglês🚀")

# --- Obter o Tópico do Usuário ---
gerar_flashcard = "Crie Flashcards formatados"

print(f'Gerando seus Flashcards')
pesquisa = agente_buscador(gerar_flashcard, data_de_hoje)
print("\n--- Resultado do agente 1 (flashcard) ---\n")
display(to_markdown(f"🟢 {pesquisa}"))
print("----------------------------------------")

🚀 Iniciando o Sistema de Criação de Flash Cards em Inglês🚀
Gerando seus Flashcards

--- Resultado do agente 1 (flashcard) ---



> 🟢 Ok, aqui está uma lista de exemplos de palavras em inglês formatadas para flashcards, criada em 17 de maio de 2025:
> 
> 
> Palavra em Inglês: |hello| Pronúncia: |həˈloʊ| Palavra em Português: |olá|
> 
> Palavra em Inglês: |dog| Pronúncia: |dɔːɡ| Palavra em Português: |cachorro|
> 
> Palavra em Inglês: |cat| Pronúncia: |kæt| Palavra em Português: |gato|
> 
> Palavra em Inglês: |house| Pronúncia: |haʊs| Palavra em Português: |casa|
> 
> Palavra em Inglês: |car| Pronúncia: |kɑːr| Palavra em Português: |carro|
> 
> Palavra em Inglês: |tree| Pronúncia: |triː| Palavra em Português: |árvore|
> 
> Palavra em Inglês: |sun| Pronúncia: |sʌn| Palavra em Português: |sol|
> 
> Palavra em Inglês: |moon| Pronúncia: |muːn| Palavra em Português: |lua|
> 
> Palavra em Inglês: |water| Pronúncia: |ˈwɔːtər| Palavra em Português: |água|
> 
> Palavra em Inglês: |fire| Pronúncia: |ˈfaɪər| Palavra em Português: |fogo|
> 
> Palavra em Inglês: |food| Pronúncia: |fuːd| Palavra em Português: |comida|
> 
> Palavra em Inglês: |friend| Pronúncia: |frɛnd| Palavra em Português: |amigo|
> 
> Palavra em Inglês: |family| Pronúncia: |ˈfæməli| Palavra em Português: |família|
> 
> Palavra em Inglês: |happy| Pronúncia: |ˈhapi| Palavra em Português: |feliz|
> 
> Palavra em Inglês: |sad| Pronúncia: |sæd| Palavra em Português: |triste|
> 
> Palavra em Inglês: |big| Pronúncia: |bɪɡ| Palavra em Português: |grande|
> 
> Palavra em Inglês: |small| Pronúncia: |smɔːl| Palavra em Português: |pequeno|
> 
> Palavra em Inglês: |good| Pronúncia: |ɡʊd| Palavra em Português: |bom|
> 
> Palavra em Inglês: |bad| Pronúncia: |bæd| Palavra em Português: |mau|
> 
> Palavra em Inglês: |yes| Pronúncia: |jɛs| Palavra em Português: |sim|
> 
> Palavra em Inglês: |no| Pronúncia: |noʊ| Palavra em Português: |não|
> 
> Palavra em Inglês: |hot| Pronúncia: |hɒt| Palavra em Português: |quente|
> 
> Palavra em Inglês: |cold| Pronúncia: |koʊld| Palavra em Português: |frio|
> 


----------------------------------------


#🗒️Agente que cria Qualquer Resumo para você🗒️

In [ ]:
print("😀Olá sou seu agente Pessoal, me diga quais assunto você tem interesse em receber um resumão ?😀")
print("\n\n")
entrada = input('Você: ')
while entrada != 'sair':

  resposta = agente_professor(entrada)
  print("\n--- ⚡resultado do agente de resumos⚡ ---\n")
  display(to_markdown(f"🤖: {resposta}"))
  print("\n digite 'sair' para se despedir👋")
  print("----------------------------------------")
  entrada = input('Você: ')

😀Olá sou seu agente Pessoal, me diga quais assunto você tem interesse em receber um resumão ?😀




--- ⚡resultado do agente de resumos⚡ ---



> 🤖: Com prazer! Para criar um resumo preciso da Lei número 1 do livro "A Lei do Triunfo", preciso primeiro entender qual é essa lei específica. Poderia, por favor, me informar qual é a Lei número 1 conforme apresentada no livro? Se você não tiver essa informação, posso pesquisar na internet para encontrar a resposta e, em seguida, fornecer um resumo detalhado.
> 
> De acordo com as fontes, a Lei número 1 do livro "A Lei do Triunfo" de Napoleon Hill é o conceito de "Master Mind".
> 
> **Resumo da Lei do Master Mind:**
> 
> *   **O que é:** A Lei do Master Mind define que, para alcançar objetivos grandiosos, é essencial associar-se a outras pessoas que compartilham da mesma linha de pensamento. A união de duas ou mais mentes cria uma força maior do que a soma de suas partes individuais.
> *   **Interdependência:** Reconhece a importância da colaboração e do apoio mútuo.
> *   **Harmonia:** Enfatiza a importância de trabalhar em rapport (empatia) com seus sócios, buscando pessoas que seguem uma mesma linha de pensamento.
> *   **Benefícios:** A união de mentes gera um "todo" maior do que a soma das partes, potencializando o sucesso que individualmente não seria possível.
> 
> Em essência, a Lei do Master Mind ensina que o sucesso raramente é alcançado sozinho e que a colaboração estratégica com outras pessoas é fundamental para atingir objetivos maiores.
> 



 digite 'sair' para se despedir👋
----------------------------------------


#📚Agente que cria Roteiro de Estudos para você📚

In [ ]:
print("😀Olá sou seu agente Pessoal, me diga quais assunto você tem interesse em receber um roteiro de estudos ?😀")
print("\n\n")
entrada = input('Você: ')
while entrada != 'sair':

  resposta = agente_estudo(entrada)
  print("\n--- 📚resultado do agente de estudo📚 ---\n")
  display(to_markdown(f"🤖: {resposta}"))
  print("\n digite 'sair' para se despedir👋")
  print("----------------------------------------")
  entrada = input('Você: ')

😀Olá sou seu agente Pessoal, me diga quais assunto você tem interesse em receber um roteiro de estudos ?😀



Você: oi

--- 📚resultado do agente de estudo📚 ---



> 🤖: Olá! Em que posso te ajudar hoje? Você gostaria de criar um roteiro de estudos? Se sim, por favor, me forneça as seguintes informações:
> 
> 1.  **Assunto de Estudo:** Qual o tema principal que você deseja estudar?
> 2.  **Objetivo Principal:** Qual o resultado que você espera alcançar ao estudar esse assunto (ex: entender os fundamentos, preparar-se para um exame, aprender a aplicar um conceito, etc.)?
> 3.  **Tempo Disponível:** Quantas horas por semana você pode dedicar a este estudo?
> 4.  **Nível de Conhecimento:** Qual seu nível atual sobre o assunto (iniciante, básico, intermediário, avançado)?
> 



 digite 'sair' para se despedir👋
----------------------------------------
Você: redes, entender os fundamentos, 4 horas, iniciante

--- 📚resultado do agente de estudo📚 ---



> 🤖: Roteiro de Estudos Inicial: Redes de Computadores
> 
> Objetivo: Entender os fundamentos de redes.
> 
> Tempo Semanal Dedicado: 4 horas.
> 
> Nível de Conhecimento: Iniciante.
> 
> Estrutura Sugerida:
> 
> Semana 1:
> 
> Tópico 1: Introdução a Redes de Computadores.
> Foco: O que são redes, por que são importantes, tipos de redes (LAN, WAN, etc.), componentes básicos (servidores, clientes, roteadores, switches).
> Recursos Sugeridos: Artigos introdutórios online, vídeos explicativos no YouTube, capítulos iniciais de livros sobre redes.
> Tempo Estimado: 2 horas.
> Semana 2:
> 
> Tópico 2: Modelo OSI e Modelo TCP/IP.
> Foco: Compreender a estrutura em camadas das redes, as funções de cada camada e como os dados são transmitidos entre elas.
> Recursos Sugeridos: Diagramas explicativos, descrições detalhadas das camadas em livros e artigos.
> Tempo Estimado: 2 horas.
> Semana 3:
> 
> Tópico 3: Endereçamento IP e Protocolo IP.
> Foco: Como os dispositivos são identificados em uma rede, o que são endereços IPv4 e IPv6, como funciona o protocolo IP para roteamento de dados.
> Recursos Sugeridos: Tutoriais sobre endereçamento IP, vídeos demonstrativos de configuração de redes.
> Tempo Estimado: 2 horas.
> Semana 4:
> 
> Tópico 4: Protocolos de Transporte: TCP e UDP.
> Foco: As diferenças entre TCP (conexão confiável) e UDP (conexão não confiável), quando usar cada um e como eles funcionam.
> Recursos Sugeridos: Explicações comparativas entre TCP e UDP, exemplos de aplicações que usam cada protocolo.
> Tempo Estimado: 2 horas.
> Semanas 5 e 6:
> 
> Tópico 5: Meios de Transmissão e Topologias de Rede.
> Foco: Tipos de cabos (par trançado, fibra óptica), tecnologias sem fio (Wi-Fi), como os dispositivos são organizados em uma rede (estrela, anel, etc.).
> Recursos Sugeridos: Artigos sobre diferentes tipos de cabos e suas características, diagramas de topologias de rede.
> Tempo Estimado: 4 horas (2 horas por semana).
> Revisão: Recomenda-se revisar os tópicos aprendidos ao final de cada semana/período.
> 
> Observação: Este é um roteiro inicial. Ajuste os tópicos, recursos e tempo de acordo com sua necessidade e progresso.



 digite 'sair' para se despedir👋
----------------------------------------
Você: sair


#♟️Agente Especialista em Lógica de Programação (Pythom)🐍

In [ ]:
print("🤖Olá sou seu agente Pessoal, de Lógica de Programação Python ?🤖")
print("\n\n")
entrada = input('Você: ')
while entrada != 'sair':

  resposta = agente_python(entrada)
  print("\n--- ⚡resultado do agente de lógica⚡ ---\n")
  display(to_markdown(f"🟢  {resposta}"))
  print("\n digite 'sair' para se despedir👋")
  print("----------------------------------------")
  entrada = input('Você: ')